In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv(r"C:\Users\fatih\OneDrive\Desktop\car_price_prediction.csv")

# "-" olan değerleri ortalama ile doldur
df['Levy'].replace('-', pd.NA, inplace=True)
df['Levy'].fillna(df['Levy'].dropna().astype(float).mean(), inplace=True)
df['Levy'] = df['Levy'].astype(float)

# Kategorik ve sayısal sütunları bul ve Kategorik sütunları etiket kodla
obj_col = ["Manufacturer", "Model", "Category", "Leather interior", "Fuel type", "Engine volume", "Gear box type", "Drive wheels", "Doors", "Wheel", "Color"]
le = LabelEncoder()
for col in obj_col:
    df[col] = le.fit_transform(df[col])

# Sayısal sütunları normalize et
num_col = ["Levy", "Prod. year", "Mileage", "Cylinders", "Airbags"]
norm = Normalizer()
df[num_col] = norm.fit_transform(df[num_col])

# Veriyi eğitim ve test setlerine ayır
X = df.drop(columns=["ID", "Price"])
y = df['Price']

# Lojistik regresyon modelini oluştur ve eğit
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
log_reg = LogisticRegression()
fit_model = log_reg.fit(X_train, y_train)

# Tahminler yap
preds = fit_model.predict(X_test)
probs = fit_model.predict_proba(X_test)

# Sonuçları bir veri çerçevesine dönüştür
model_results = pd.DataFrame({
    'PredictedClass': preds,
    'TrueClass': y_test,
    'ClassProb:0': probs[:, 0],
    'ClassProb:1': probs[:, 1]
})

# Doğru sınıflandırma sayısını hesapla
true_count = sum(preds == y_test)
print("Number of True Classifications =", true_count)
print("Accurate Classification Ratio =", true_count / len(y_test))